
# Repaso de conceptos clave

- **Secuencias homólogas**: similitud por ancestría común
- **Alineamiento** global vs local: ordenamiento de secuencias según similitud. **Global**: utiliza toda la secuencia. **Local**: utiliza los fragmentos más similares
- **Secuencias conservadas**: aquellas que comparten nucleótidos o aminoacidos en la misma posición (opuesto en divergentes)
- **Score**: parámetro que indica la calidad del alineamiento
- **Dendograma** (Cladograma): hipótesis sobre relación evolutiva de secuencias

Para el taller entregable, realizamos un alineamientos de diferentes hospederos. Ahora vamos a visualizar la relación filogenética de estas secuencias. Para esto entramos a Apolo y cargamos el módulo de python y el ambiente de biopython. Vamos a utilizar la librería [Phylo](https://biopython-tutorial.readthedocs.io/en/latest/notebooks/13%20-%20Phylogenetics%20with%20Bio.Phylo.html)

In [ ]:
ssh lsalazarj@apolo.eafit.edu.co
module load python/3.6.0_miniconda-4.3.11_gcc-11.2.0
conda activate biopython
python3.11

In [ ]:
from Bio import Phylo
tree = Phylo.read("spikes_hosts.dnd","newick")
Phylo.draw_ascii(tree)
Phylo.write(tree, "spikes_hosts.nwk", "newick")

["Newick"](https://evolution.genetics.washington.edu/phylip/newicktree.html) es un formato para escribir árboles filogenéticos de forma simplificada. 

<img src="newicktree.png"/>

Este árbol se repesentaría con el siguiente texto:
(B,(A,C,E),D);

Para este ejercicio, las accesiones de las secuencias fueron definidas. Ahora vamos a realizar la búsqueda de las secuencias mas similares de todas las disponibles en la base de datos usando Basic Local Alignment Search Tool, BLAST (Altschul et al. 1997; Altschul et al. 1990)

# BLAST

La búsqueda de similitudes es un primer paso para encontrar  información útil sobre funcionalidad y/o ancestría. 

A la hora de buscar similitud en bases de datos extensas, los algoritmos de programación dinámica (como Needleman-Wunsch y Smith-Waterman) son ineficientes. Las herramientas mas eficientes son heurísticas: "atajos" algoritmicos que no producen una solición óptima global, pero que son mucho más rápidos y eficientes.


Antes de realizar una búsqueda en BLAST, se genera un índice que consiste de *palabras* de una longitud *k*, las cuales se denominan **k-mers**. Éste índice se almacena en la memoria RAM del computador, así que BLAST puede ser muy rápido buscando estas palabras. Para inciarl un búsqueda por similitud, el algoritmo de BLAST primero busca en la secuencia consulta (*query*). usando la base de datos indexada, las secuencias que contengan un score alto de k-mers son escogidas. De allí se realiza un alineamiento local alreadedor de estos k-mers.

<img src="blast_glossary-Image001.jpeg"/>

### Tipos de BLAST

De acuerdo a la búsqueda que necesitemos hay diferentes tipos de [blast](https://desmid.github.io/mview/manual/blast/blast2.html):

- blastn: búsqueda de nucleótidos en base de datos de nucleótidos
- blastp: búsqueda de proteína en base de datos de proteínas
- blastx: búsqueda de nucleótidos en base de datos de proteínas, traduciendo la secuencia query en diferentes marcos de lectura 
- tblastn: búsqueda de proteínas en base de datos de nucleótidos, traduciendo los hit en diferentes marcos de lectura 
- tblastx: búsqueda de nucleótidos en base de datos de nucleótidos, traduciendo query y hit en diferentes marcos de lectura 
- psiblast: Position Specific Iterated-BLAST (PSI-BLAST) es basado en blastp pero utiliza los perfiles de conservación para hallar secuencias distantemente relacionadas


### E-value (Expectation-values)

Cuando realizamos una búsqueda por similitud, es importante identificar si la secuencia target es significativamente similar a la query. Es enste caso la significancia significa que el score del alineamiento (S)
 es mayor que uno esperado al azar. Un hit siempre va a tener algún tipo de similitud, y la probabilidad de encontrar un score alto aumenta con una secuencia query mas larga (m) y/o una base de datos mas grande (n). Para cuantificar la significancia de un hit, calculamos el E-value (E), que representa el número de hits esperados con un score mínimo de *S*, usando una secuencia query de longitud *m*, y una búsqueda aleatoria en una base de datos n: 

$$
 E = K * m *n * e^{lamda S}
$$

 Algunas reglas no formales ("rules of thumb") que pueden servir como guía para considerar la significancia de los hit:

- $E-value < 10e^{-100}$ Secuencias idénticas. Esperarías un alineamiento largo y completo entre la secuecnia de consulta y hit
- $10e^{-50} < E-value < 10e^{-100}$ Secuencias casi idénticas. Esperarías un fragmento largo de alineamiento entre la secuecnia de consulta y hit
- $10e^{-10} < E-value < 10e^{-50}$ Secuencias cercanas o un dominio
- $1 < E-value < 10e^{-6}$ Podría ser un homólogo, pero es un área gris
- $E-value > 1$ Las proteínas probablemente no están relacionados
- $E-value > 10$ Los hits son irrelevantes o la secuencia consulta muy pequeña

## Búsqueda de la espícula de SARS-CoV con BLAST

Vamos a volver a las secuencias que habíamos descargado de SARS-CoV y a tomar la secuencia de la proteína de la espícula ("glycoproteins"). Vamos hacer un blast con ella para buscar secuencias homólogas. Que tipo de blast utilizaría?  

Existen dos grandes plataformas que presetan el servicio:
- NCBI: https://blast.ncbi.nlm.nih.gov/Blast.cgi
- EBI: https://www.ebi.ac.uk/Tools/sss/

Hoy vamos a trabajar con la de NCBI

<div class="alert alert-block alert-info">
<b>Ejercicio de BLAST</b> Escoja a opción adecuada pegue la secuencia en el campo de la secuencia query. Blast contiene muchas opciones con buena documentiación. Escoja una (o experimente con varias) y luego presiones "Blast". Los resultados se pueden obtener en forma visual o tabulada. 
*Tip*: Excluya virus del SARS-CoV humano y constructos sintéticos porque estos saturan la búsqueda
 </div>

Cuantos hits obutvo? Cuales son los rangos de E-values que obtuvo? Qué descripciones tienen las secuencias y a que hosperedos pertencen?

<div class="alert alert-block alert-info">
<b>Ejercicio Evaluable</b> 

  1) Descargue por lo menos 3 secuencias correspondientes a diferentes hospederos
  2) Obtenga un alineamiento y un dendograma  (como lo hicimos al inicio) con la secuencia consulta y hits que obtuvo
  3) Repita el ejercicio de Blast, esta vez con la secuencia de nucleótidos codificadores (cds)
  2) Obtenga un alineamiento y un dendograma con los 3 mejores hits
  3) Que diferencias observa con respecto al alineamiento de proteínas?
  4) Escriba un breve reporte (una página) sobre las similitudes y diferencias 
  </div>